<a href="https://colab.research.google.com/github/sarkarj2002/UCB-Assignment-17.1/blob/main/prompt_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [ ]:
import pandas as pd
import numpy as np
import time
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC


In [ ]:
df = pd.read_csv('bank-additional-full.csv', sep = ';')
df.isnull().sum()
print(df.head())
df1=pd.read_csv('bank-additional.csv', sep = ';')
df1.isnull().sum()
print(df1.head())
df1.shape


   age        job  marital    education  default housing loan    contact  \
0   56  housemaid  married     basic.4y       no      no   no  telephone   
1   57   services  married  high.school  unknown      no   no  telephone   
2   37   services  married  high.school       no     yes   no  telephone   
3   40     admin.  married     basic.6y       no      no   no  telephone   
4   56   services  married  high.school       no      no  yes  telephone   

  month day_of_week  ...  campaign  pdays  previous     poutcome emp.var.rate  \
0   may         mon  ...         1    999         0  nonexistent          1.1   
1   may         mon  ...         1    999         0  nonexistent          1.1   
2   may         mon  ...         1    999         0  nonexistent          1.1   
3   may         mon  ...         1    999         0  nonexistent          1.1   
4   may         mon  ...         1    999         0  nonexistent          1.1   

   cons.price.idx  cons.conf.idx  euribor3m  nr.employed

FileNotFoundError: [Errno 2] No such file or directory: 'bank-additional.csv'

In [ ]:
df.head()
print(df.columns)
print(df1.columns)



Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')


NameError: name 'df1' is not defined

### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [ ]:
# The primary business objective is to accurately identify and target customers who are most likely
# subscibe to the bank products following a telephone marketing capaign.by analyzing customer demographics,
# previous interactions, and campaign details, the goal is to develop predictive modelsthat enable the bank
# to optimize  marketing efforts, improve conversion rates, and allocate resource more efficiently. This will
# help the bank increase product adoption, reduce marketing costs, and enhance customer satisfaction by focusing
# outreach on individuals with the highest likelihood of responding positively.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [ ]:
# Assume y column is the target and all other columns are features
X=df.drop('y',axis=1)
y=df['y']
# Preprocessing (encoding categorical variables if needed)
X=pd.get_dummies(X)



### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [ ]:
# The most common baseline  for clssification tasks is the accuracy of a simple model that always predicts
# the majority class (ie the class that occurs most frequently in the dataset)
# For example, if 85% of customer in the dataset did not subscribe the product, a model that always predicts
# "no subscription" would have an accuracy of 85%
# The classifiers that we would build should aim to beat the baseline accuracy. However if the data is imbalanced
# (far more 'no' that 'yes' responses) also consider baseline metrics for precision, recall, and F1 score for the minority class ('yes)


In [ ]:
maj_class=y.value_counts().idxmax()
baseline_accuracy=(y==maj_class).mean()
print(baseline_accuracy)


0.8873458288821987


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [ ]:
lr=LogisticRegression(max_iter=10000)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred,pos_label='yes'))
print(recall_score(y_test, y_pred,pos_label='yes'))
print(f1_score(y_test, y_pred,pos_label='yes'))

0.9101723719349356
0.662771285475793
0.42459893048128344
0.5176010430247718


### Problem 9: Score the Model

What is the accuracy of your model?

In [ ]:
# 91%

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
models = {
    'Logistic Regression': LogisticRegression(max_iter=10000),
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'SVM': SVC()
}
results = [] # Initialize as a list
for name, model in models.items():
    start_time = time.time() # Capture start time before fitting
    if name in ['KNN','SVM']:
      model.fit(X_train_scaled, y_train)
    else:
      model.fit(X_train, y_train)
    end_time = time.time() # Capture end time after fitting
    fit_duration = end_time - start_time # Calculate fit duration

    # Make predictions
    if name in ['KNN','SVM']:
      train_pred=model.predict(X_train_scaled)
      test_pred=model.predict(X_test_scaled)
    else:
      train_pred=model.predict(X_train)
      test_pred=model.predict(X_test)

    train_accuracy = accuracy_score(y_train, train_pred)
    test_accuracy = accuracy_score(y_test, test_pred)
    results.append({
        'Model': name,
        'Train Accuracy': train_accuracy,
        'Test Accuracy': test_accuracy,
        'Fit Time (s)': fit_duration # Use the calculated duration
    })

results_df=DataFrame(results) # Create DataFrame once after the loop
print(results_df)

                 Model  Train Accuracy  Test Accuracy  Fit Time (s)
0  Logistic Regression        0.911533       0.910172    163.492618
1                  KNN        0.919879       0.898155      0.047372
2        Decision Tree        1.000000       0.887837      0.377787
3                  SVM        0.928134       0.909080     41.203136


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.


- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [ ]:
from sklearn.model_selection import GridSearchCV
dt_params={
    'max_depth' : [3,5,7,10,None],
    'min_samples_split' : [2,5,10],
}
dt_grid=GridSearchCV(DecisionTreeClassifier(),dt_params,cv=5,scoring='accuracy')
dt_grid.fit(X_train,y_train)
print("Best Decision Tree Parameters:", dt_grid.best_params_)
print("Best Decision Tree CV Score:",dt_grid.best_score_)
knn_params = {
    'n_neighbors': [3, 5, 7, 10],
    'weights' : ['uniform', 'distance']
}
knn_grid = GridSearchCV(KNeighborsClassifier(), knn_params, cv=5, scoring='accuracy')
knn_grid.fit(X_train_scaled, y_train)
print("Best KNN Parameters:", knn_grid.best_params_)
print("Best KNN CV Score:",knn_grid.best_score_)


Best Decision Tree Parameters: {'max_depth': 5, 'min_samples_split': 5}
Best Decision Tree CV Score: 0.91350531107739
Best KNN Parameters: {'n_neighbors': 10, 'weights': 'distance'}
Best KNN CV Score: 0.898300455235205


In [ ]:
lr_params = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'],
    'solver': ['liblinear','ibfgs']
}
lr_grid = GridSearchCV(LogisticRegression(max_iter=10000), lr_params, cv=5, scoring='accuracy')
lr_grid.fit(X_train, y_train)
print("Best Logistic Regression Parameters:", lr_grid.best_params_)
print("Best Logistic Regression CV Score:",lr_grid.best_score_)


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/_

Best Logistic Regression Parameters: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
Best Logistic Regression CV Score: 0.9101972685887709


In [ ]:
svm_params = {
    'C': [0.1, 1,10,100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}
svm_grid = GridSearchCV(SVC(), svm_params, cv=5, scoring='accuracy')
svm_grid.fit(X_train_scaled, y_train)
print("Best SVM Parameters:", svm_grid.best_params_)
print("Best SVM CV Score:",svm_grid.best_score_)

In [ ]:
svm_params = {
    'C': [0.1, 1],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}
svm_grid = GridSearchCV(SVC(), svm_params, cv=5, scoring='accuracy')
svm_grid.fit(X_train_scaled, y_train)
print("Best SVM Parameters:", svm_grid.best_params_)
print("Best SVM CV Score:",svm_grid.best_score_)

NameError: name 'GridSearchCV' is not defined

In [ ]:
df1=pd.read_csv('bank-additional.csv', sep = ';')
df1.isnull().sum()
print(df1.head())
df1.shape

   age          job  marital          education default  housing     loan  \
0   30  blue-collar  married           basic.9y      no      yes       no   
1   39     services   single        high.school      no       no       no   
2   25     services  married        high.school      no      yes       no   
3   38     services  married           basic.9y      no  unknown  unknown   
4   47       admin.  married  university.degree      no      yes       no   

     contact month day_of_week  ...  campaign  pdays  previous     poutcome  \
0   cellular   may         fri  ...         2    999         0  nonexistent   
1  telephone   may         fri  ...         4    999         0  nonexistent   
2  telephone   jun         wed  ...         1    999         0  nonexistent   
3  telephone   jun         fri  ...         3    999         0  nonexistent   
4   cellular   nov         mon  ...         1    999         0  nonexistent   

  emp.var.rate  cons.price.idx  cons.conf.idx  euribor3m  nr.e

(4119, 21)

In [ ]:
from sklearn.model_selection import GridSearchCV
df1=pd.read_csv('bank-additional.csv', sep = ';')
df1.isnull().sum()
X=df1.drop('y',axis=1)
y=df1['y']
X=pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
df1=pd.read_csv('bank-additional.csv', sep = ';')
df1.isnull().sum()
X=df1.drop('y',axis=1)
y=df1['y']
X=pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm_params = {
    'C': [0.1, 1,10,100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}
svm_grid = GridSearchCV(SVC(), svm_params, cv=5, scoring='accuracy')
svm_grid.fit(X_train_scaled, y_train)
print("Best SVM Parameters:", svm_grid.best_params_)
print("Best SVM CV Score:",svm_grid.best_score_)

Best SVM Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
Best SVM CV Score: 0.9119878603945372


##### Questions